In [2]:
from ipywidgets import interact, widgets
import pandas as pd
import requests
import numpy as np
from IPython.core.display import display, HTML
from pandas_cache import pd_cache, timeit

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot

from bokeh.models.tools import HoverTool
import pandas_bokeh

pandas_bokeh.output_notebook()

FORMAT = '%Y-%m-%d'
output_notebook()

Loading BokehJS ...

Loading BokehJS ...

## API SETUP

This Jupyter notebook pulls data from AlphaVantage.<br /> 
If you don't have one, claim your free API Key at https://www.alphavantage.co/support/#api-key

In [5]:
@interact.options(manual=True)
def set_api(text="Enter your alphavantage API KEY"):
    global API 
    API = text
    print(f"Your API key is: '{text}'")

interactive(children=(Text(value='Enter your alphavantage API KEY', description='text'), Button(description='R…

In [23]:
#@pd_cache
def stock_daily(stock):
  
    #tsleep(10)
  
    url = 'https://www.alphavantage.co/query'
    payload = {
    'function': 'TIME_SERIES_DAILY',
    'symbol': stock,
    #'interval': '5min',
    'outputsize': 'full',
    'apikey': API
    }

    try: 
        r = requests.get(url, params=payload)
        r.raise_for_status()
    except requests.exceptions.HTTPError as err: 
        print('Failed request alphavantage',err)
        return e
    df = pd.DataFrame(r.json()[list(r.json().keys())[1]]).T
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df=df.astype(float)
    df['Mid'] = df[['Open', 'Close']].mean(axis=1)
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    return df

In [7]:
def sma(df, column, sma1, sma2):
    df['sma1'] = df[column].rolling(window=sma1).mean()
    df['sma2'] = df[column].rolling(window=sma2).mean()
    return df

In [8]:
def macd(df, column='Close', fast_MA=12, slow_MA=26, period=9):
    exp1 = df[column].ewm(span=fast_MA, adjust=False).mean()
    exp2 = df[column].ewm(span=slow_MA, adjust=False).mean()
    df['macd1'] = exp1-exp2
    df['macd2'] = df.macd1.ewm(span=period, adjust=False).mean()
    return df

In [9]:
def rsi(df, column, n=14):
    delta = df[column].diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0
    RolUp = dUp.rolling(n).mean()
    RolDown = dDown.abs().rolling(n).mean()

    RS = RolUp / RolDown
    df['RSI'] = 100.0 - (100.0 / (1.0 + RS))
    return df

In [13]:
stocks = ['AMZN',
 'MSFT',
 'SHOP',
 'NDAQ',
 'SQ',
 'V',
 'CMCSA',
 'FB',
 'DIS',
 'GOOG',
 'AMD',
 'WORK',
 'LEVI',
 'VGT',
 'ULTA',
 'ACB'
]

In [31]:
macd_slider = widgets.IntRangeSlider(
    value=[12, 26],
    min=0,
    max=200,
    step=2.
)
price_columns = ['Open', 'High', 'Low', 'Close', 'Mid']

In [37]:
@interact(stock=stocks, column=price_columns, macd_range=macd_slider)
def viz_stock(stock, column,macd_range):
    df = stock_daily(stock)
    fast_MA, slow_MA = macd_range
    df = macd(df, column, fast_MA, slow_MA)
    df = rsi(df, column)
    source = ColumnDataSource(df)
    
    min_date = str(df.index.min().strftime(FORMAT))
    max_date = str(df.index.max().strftime(FORMAT))

    p1 = figure(title=f"{stock} from {min_date} to {max_date}", y_axis_type="linear",
               plot_height=400, x_axis_type="datetime", active_scroll= "wheel_zoom")
    p1.yaxis.axis_label = column
    p1.line('index', column, line_width = 1.5, source=source, line_color='blue', alpha=0.6,legend=stock)


    hover = HoverTool(
        tooltips=[('date', '@index{%F}'), (column, f'$@{column}{{0,0.00}}'), ('volume', '@Volume{0.00 a}'),
                  ('sma1', '@sma1{0,0}'), ('sma2', '@sma2{0,0}'),
                  ('macd1', '@macd1{0.00}')
                 ],
        formatters={'index': 'datetime'},
        mode='vline'
    )

    p1.add_tools(hover)
    p1.legend.location = "top_left"

    
    p2 = figure(title=f"MACD: 12,26,9", y_axis_type="linear",
                   plot_height=200, x_axis_type="datetime",
                   x_range=p1.x_range, active_scroll= "wheel_zoom")
    p2.line('index', 'macd1', line_width = 1, source=source, line_color='red',legend='MACD1')
    p2.line('index', 'macd2', line_width = 1, source=source, line_color='green',legend='MACD2')
    p2.add_tools(hover)
    p2.legend.location = "top_left"
    
    p3 = figure(title=f"RSI", y_axis_type="linear",
                   plot_height=150, x_axis_type="datetime",
                   x_range=p1.x_range, active_scroll= "wheel_zoom")
    p3.line('index', 'RSI', line_width = 1, source=source, line_color='purple',legend='rsi')
    p3.add_tools(hover)
    p3.legend.location = "top_left"
    
    f = gridplot([[p1],[p2], [p3]], plot_width=1000)
    show(f);

interactive(children=(Dropdown(description='stock', options=('AMZN', 'MSFT', 'SHOP', 'NDAQ', 'SQ', 'V', 'CMCSA…